In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-07-15 16:20:59--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-15 16:20:59 (45.5 MB/s) - ‘minsearch.py.1’ saved [3832/3832]



In [2]:
import minsearch

In [3]:
import json

In [4]:
with open('documents.json','rt') as f_in:
    docs_raw=json.load(f_in)

In [5]:
documents = []

for course in docs_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index=minsearch.Index(text_fields=["question","text","section"],
                keyword_fields=["course"])

In [19]:
query="This course has been already started, can i enroll?"

In [20]:
index.fit(documents)

In [21]:
boost={"question":3.0,"section":0.5}
results=index.search(query=query,
                     boost_dict=boost,
                     num_results=5)
             

In [23]:
results

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for t

In [12]:
! pip install -q -U google-generativeai

In [13]:
import google.generativeai as genai

In [14]:
model=genai.GenerativeModel('gemini-pro')

In [15]:
response=model.generate_content("essay on global warming?")

In [16]:
response.text

"**Global Warming: A Pressing Issue for Humankind**\n\nGlobal warming, a phenomenon characterized by the rise in average global temperatures, has emerged as one of the most pressing environmental challenges of our time. This essay will delve into the causes, effects, and potential solutions to global warming, highlighting its profound implications for humankind.\n\n**Causes of Global Warming**\n\nThe primary cause of global warming is the increased emission of greenhouse gases (GHGs) into the atmosphere. Human activities, such as burning fossil fuels for transportation and electricity generation, contribute to the release of carbon dioxide (CO2), methane (CH4), and nitrous oxide (N2O). These gases trap heat in the Earth's atmosphere, leading to a gradual increase in global temperatures.\n\n**Effects of Global Warming**\n\nGlobal warming has far-reaching effects on both natural and human systems. Rising temperatures result in:\n\n* **Extreme Weather Events:** Increased frequency and int

In [17]:
prompt_template="""
you are a course teaching assistant. Answer the QUESTION based on the CONTEXT.
use only the facts from the CONTEXT when answering the QUESTION.
If the context does not have the answer, output None.

QUESTION: {question}
CONTEXT : {context}
"""

In [25]:
context=""
for doc in results:
    context= context+f"section: {doc['section']}\nquestion:{doc['question']}\nanswer:{doc['text']}\n\n"

In [27]:
print(context)

section: General course-related questions
question:The course has already started. Can I still join it?
answer:Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.

section: General course-related questions
question:Course - What are the prerequisites for this course?
answer:GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites

section: General course-related questions
question:Course - Can I follow the course after it finishes?
answer:Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you

In [35]:
prompt=prompt_template.format(question="This course has been already started, can i enroll?",context=context)

In [36]:
response=model.generate_content(prompt)

In [37]:
response.text

'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.'